In [17]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import pandas as pd

In [18]:
# Load mô hình và tokenizer
model = AutoModelForSequenceClassification.from_pretrained("./bert_finetuned")
tokenizer = AutoTokenizer.from_pretrained("./bert_finetuned")

In [19]:
# Load MultiLabelBinarizer để giải mã nhãn
mlb = MultiLabelBinarizer()
mlb.classes_ = np.array(['fiction', 'romance', 'young adult', 'fantasy', 'contemporary'])

In [20]:
# Hàm inference
def predict_tags(descriptions, threshold=0.5):
    """
    Dự đoán nhãn cho một hoặc nhiều đoạn mô tả.
    
    Args:
        descriptions (Union[str, List[str]]): Một đoạn mô tả hoặc danh sách các mô tả.
        threshold (float): Ngưỡng dự đoán cho các nhãn.
        
    Returns:
        List[List[str]]: Danh sách các nhãn dự đoán cho mỗi mô tả.
    """
    if isinstance(descriptions, str):
        descriptions = [descriptions]
    
    # Tokenize mô tả
    inputs = tokenizer(descriptions, truncation=True, padding=True, max_length=128, return_tensors="pt")
    
    # Dự đoán logits
    model.eval()
    with torch.no_grad():
        logits = model(**inputs).logits
    
    # Áp dụng sigmoid để tính xác suất
    probs = torch.sigmoid(logits).numpy()
    
    # Lấy nhãn dựa trên ngưỡng
    predicted_labels = [
        [mlb.classes_[i] for i, prob in enumerate(prob_vector) if prob > threshold]
        for prob_vector in probs
    ]
    
    return predicted_labels

In [21]:
# Đọc dữ liệu test từ file CSV
test_data = pd.read_csv("../Data/Train_and_Test_dataset/test.csv")

In [22]:
# Danh sách lưu mẫu
correct_samples = []
correct_predictions = []
incorrect_samples = []
incorrect_predictions = []

for i, row in test_data.iterrows():
    # Lấy thông tin mô tả và nhãn thực tế
    description = row["description"]
    true_labels = set(row["genres"].split(","))
    
    # Dự đoán
    predicted_labels = set(predict_tags(description, threshold=0.5)[0])
    
    # Phân loại đúng/sai
    if true_labels == predicted_labels:
        if len(correct_samples) < 3:  # Chỉ lưu tối đa 3 mẫu đúng
            correct_samples.append(row)
            correct_predictions.append(predicted_labels)
    else:
        if len(incorrect_samples) < 2:  # Chỉ lưu tối đa 2 mẫu sai
            incorrect_samples.append(row)
            incorrect_predictions.append(predicted_labels)
    
    # Dừng nếu đủ 5 mẫu
    if len(correct_samples) == 3 and len(incorrect_samples) == 2:
        break

if correct_samples or incorrect_samples:
    correct_df = pd.DataFrame(correct_samples)
    correct_df["Predicted Tags"] = correct_predictions
    
    incorrect_df = pd.DataFrame(incorrect_samples)
    incorrect_df["Predicted Tags"] = incorrect_predictions

    print("Correct Predictions:")
    print(correct_df[["description", "genres", "Predicted Tags"]])
    
    print("\nIncorrect Predictions:")
    print(incorrect_df[["description", "genres", "Predicted Tags"]])

Correct Predictions:
                                           description  \
64   Sometimes, you don't need romanceSir Brynn of ...   
106  Two boys  a slow learner stuck in the body of ...   
179  Charlotte Ramsey is the new girl again. After ...   

                               genres                        Predicted Tags  
64                    romance,fantasy                    {fantasy, romance}  
106  young adult,fiction,contemporary  {contemporary, fiction, young adult}  
179  fiction,young adult,contemporary  {contemporary, fiction, young adult}  

Incorrect Predictions:
                                         description  \
0  I can see ghosts. I can talk to ghosts. And, i...   
1  Conspiracy thrillers have tackled Da Vinci, At...   

                                genres  \
0  young adult,fantasy,romance,fiction   
1                      fiction,fantasy   

                                  Predicted Tags  
0  {fantasy, romance, young adult, contemporary}  
1           